<h1 style="color:blue"> Tarefa 6.1: Coleta de dados para a calculadora de imóveis (Manipulação de Dados) </h1>

Durante as aulas, você aprendeu a utilizar Python puro, Pandas e Regex para tratamento dos dados dos imóveis coletados na etapa anterior. 

Separar valores de aluguéis, números de quartos, áreas, entre outros, pode requerer procedimentos extensos e trabalhosos.

Submeta no campo abaixo um link de seu repositório no GitHub em que conste os procedimentos realizados por você para tratar os dados dos imóveis que você coletou na etapa passada.


In [1]:
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
zonas = ['sul', 'norte', 'leste', 'oeste']
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']

In [3]:
def coleta_dados(url, zona):
  # acessar a url, coletar as informacoes e ir salvando nas listas e depois juntar as listas em uma tabela
    res = requests.get(url).content
    sopa = BeautifulSoup(res)

    rs_precos = sopa.find_all('span', {'class': 'price-tag-fraction'})
    rs_areas_quartos = sopa.find_all('ul', {'class': 'ui-search-card-attributes ui-search-item__group__element'})

    lista_precos = []
    lista_areas_quartos = []
    lista_zonas = []
    
    for preco, area_quarto in zip(rs_precos, rs_areas_quartos):
        lista_precos.append(preco.text)
        lista_areas_quartos.append(area_quarto.text)
        lista_zonas.append(zona)
    
    dados_dic = {'aluguel': lista_precos,
                  'area_quarto': lista_areas_quartos,
                 'zona': lista_zonas
                  }
    
    df = pd.DataFrame(dados_dic)

    return df

In [4]:
lista_dfs = []

for zona in zonas:
  # print('Zona:', zona)
  for pagina in paginas:
    # print('Pagina:', pagina)
    url = f'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-{zona}/{pagina}'
    print('Coletando dados de', url)
    df = coleta_dados(url, zona) # retorna o df 
    lista_dfs.append(df)
    sleep(2)

Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_193
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_241
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-norte/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-p

In [27]:
# cocantenando o retorno do ML (linha) para formar um DF (axis = 0 )
df_ml = pd.concat(lista_dfs, axis=0)

In [28]:
df_ml.head(10)

,aluguel,area_quarto,zona
0,900,58 m² construídos2 quartos,sul
1,1.500,68 m² construídos2 quartos,sul
2,1.320,87 m² construídos2 quartos,sul
3,2.500,100 m² construídos2 quartos,sul
4,1.290,38 m² construídos1 quarto,sul
5,1.550,72 m² construídos2 quartos,sul
6,620,55 m² construídos1 quarto,sul
7,1.400,72 m² construídos2 quartos,sul
8,1.350,49 m² construídos2 quartos,sul
9,4.500,88 m² construídos3 quartos,sul


In [26]:

df_ml['aluguel'].replace('.', ',')
df_ml.head()

,aluguel,area_quarto,zona
0,900.00,58 m² construídos2 quartos,sul
1,1.50,68 m² construídos2 quartos,sul
2,1.32,87 m² construídos2 quartos,sul
3,2.50,100 m² construídos2 quartos,sul
4,1.29,38 m² construídos1 quarto,sul


In [29]:
#Conversão da coluna aluguel para o tipo float

df_ml['aluguel'] =df_ml["aluguel"].astype(float).replace('.', ',')
df_ml

,aluguel,area_quarto,zona
0,900.000,58 m² construídos2 quartos,sul
1,1.500,68 m² construídos2 quartos,sul
2,1.320,87 m² construídos2 quartos,sul
3,2.500,100 m² construídos2 quartos,sul
4,1.290,38 m² construídos1 quarto,sul
...,...,...,...
43,3.500,79 m² construídos2 quartos,oeste
44,3.200,65 m² construídos2 quartos,oeste
45,3.800,46 m² construídos1 quarto,oeste
46,3.909,45 m² construídos1 quarto,oeste


In [8]:
# Crianção de 4 novas colunas a partir da coluna area_quarto,  separado por espaços

df_novo = df_ml["area_quarto"].str.split(" ", n = 4, expand = True)
df_novo


,0,1,2,3
0,58,m²,construídos2,quartos
1,68,m²,construídos2,quartos
2,87,m²,construídos2,quartos
3,100,m²,construídos2,quartos
4,38,m²,construídos1,quarto
...,...,...,...,...
43,79,m²,construídos2,quartos
44,65,m²,construídos2,quartos
45,46,m²,construídos1,quarto
46,45,m²,construídos1,quarto


In [9]:
#Deletar as colunas 1 e 3

del df_novo[1]
del df_novo[3]
df_novo.head()

,0,2
0,58,construídos2
1,68,construídos2
2,87,construídos2
3,100,construídos2
4,38,construídos1


In [10]:
# Renomear a coluna 0 por Area (m2)

df_novo = df_novo.rename({0:"Area (m2)"}, axis = 1)
df_novo.head()


,Area (m2),2
0,58,construídos2
1,68,construídos2
2,87,construídos2
3,100,construídos2
4,38,construídos1


In [11]:
#separar a parte numerica texto da string (construidos) 

novo2 = df_novo[2].str.replace("construídos", "")
df_novo2 = pd.DataFrame(novo2)
df_novo2.head()


,2
0,2
1,2
2,2
3,2
4,1


In [12]:
# Renomear a coluna 2 pora Quanto(s)

df_novo2 = df_novo2.rename({2:"Quarto(s)"}, axis = 1)
df_novo2.head()


,Quarto(s)
0,2
1,2
2,2
3,2
4,1


In [13]:
# Concatenar os 3 DF (df_ml, df_novo e df_novo2)

df_ml2 = pd.concat([df_ml, df_novo, df_novo2], axis = 1)
df_ml2.head()

,aluguel,area_quarto,zona,Area (m2),2,Quarto(s)
0,900.00,58 m² construídos2 quartos,sul,58,construídos2,2
1,1.50,68 m² construídos2 quartos,sul,68,construídos2,2
2,1.32,87 m² construídos2 quartos,sul,87,construídos2,2
3,2.50,100 m² construídos2 quartos,sul,100,construídos2,2
4,1.29,38 m² construídos1 quarto,sul,38,construídos1,1


In [23]:
df_ml2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1152 entries, 0 to 47
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   aluguel    1152 non-null   float64
 1   zona       1152 non-null   object 
 2   Area (m2)  1152 non-null   object 
 3   Quarto(s)  1152 non-null   object 
dtypes: float64(1), object(3)
memory usage: 45.0+ KB


In [14]:
#Deletar as colunas area_quarto e 2

del df_ml2['area_quarto']
del df_ml2[2]

df_ml2.head()

,aluguel,zona,Area (m2),Quarto(s)
0,900.00,sul,58,2
1,1.50,sul,68,2
2,1.32,sul,87,2
3,2.50,sul,100,2
4,1.29,sul,38,1


In [15]:
# converter a coluna aluguel para tipo foat

df_ml2['aluguel'] =df_ml2["aluguel"].astype(float)
df_ml2.head()

,aluguel,zona,Area (m2),Quarto(s)
0,900.00,sul,58,2
1,1.50,sul,68,2
2,1.32,sul,87,2
3,2.50,sul,100,2
4,1.29,sul,38,1


In [16]:
type(df_ml2['aluguel'].values)

numpy.ndarray

In [17]:
type(df_ml2['aluguel'].values[0])

numpy.float64

In [18]:
df_ml2.to_csv('dados_mercado_livre.csv', index=False)

In [19]:
pd.read_csv('dados_mercado_livre.csv', thousands='.')

,aluguel,zona,Area (m2),Quarto(s)
0,9000,sul,58,2
1,15,sul,68,2
2,132,sul,87,2
3,25,sul,100,2
4,129,sul,38,1
...,...,...,...,...
1147,35,oeste,79,2
1148,32,oeste,65,2
1149,38,oeste,46,1
1150,3909,oeste,45,1


# Manipulaçao de dados

In [ ]:
import pandas as pd

In [ ]:
url = 'https://raw.githubusercontent.com/cassiasamp/calculadora-de-imoveis-mar-21/main/coleta_de_dados/dados_mercado_livre.csv'
df_calc = pd.read_csv(url, thousands='.')

In [ ]:
df_calc.head()

In [ ]:
df_calc.info()

In [ ]:
df_calc.describe().T

In [ ]:
df_calc.isna().sum()